In [ ]:
import numpy as np
import pandas as pd
from nltk import word_tokenize

In [ ]:
training = pd.read_csv("0000000000002747_training_twitter_x_y_train.csv")
testing = pd.read_csv("0000000000002747_test_twitter_x_test.csv")

In [ ]:
training

In [ ]:
training.columns

In [ ]:
train = training
test = testing

In [ ]:
for x in train.columns:
    if x == 'airline_sentiment' or x=='text':
        continue
    else:
        del train[x]

In [ ]:
for x in test.columns:
    if x=='text':
        continue
    else:
        del test[x]

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
document = []
for i in range(len(train)):
    document.append((word_tokenize(train["text"][i]), train["airline_sentiment"][i]))

In [ ]:
document

In [ ]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [ ]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctutations = list(string.punctuation)
stops.update(punctutations)

In [ ]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag(w)
            clean_words = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_words.lower())
    
    return output_words

In [ ]:
document

In [ ]:
documents = []
for x in document:
    documents.append((clean_review(x[0]), x[1]))

In [ ]:
training_documents = documents


In [ ]:
all_words = []
for doc in training_documents:
    all_words += doc[0]

In [ ]:
import nltk

In [ ]:
freq = nltk.FreqDist(all_words)
common = freq.most_common(3000)
features = [i[0] for i in common]

In [ ]:
features

In [ ]:
def get_feature_dict(words):
    current_features = {}
    words_set = set(words)
    for w in features:
        current_features[w] = w in words_set
    return current_features

In [ ]:
training_data = [(get_feature_dict(doc[0]), doc[1]) for doc in training_documents]

# testing data

In [ ]:
test.head()

In [ ]:
test_data_doc = []
for i in range(len(test)):
    test_data_doc.append((word_tokenize(test["text"][i]),))
    


In [ ]:
test_data_doc

In [ ]:
test_documents = []
for x in test_data_doc:
    test_documents.append((clean_review(x[0]),))

In [ ]:
testing_data = [(get_feature_dict(doc[0]),) for doc in test_documents]

In [ ]:
testing_data[0]

In [ ]:
training_data[0]

In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=20, n_jobs=-1)
clf_rf = SklearnClassifier(rf)

In [ ]:
clf_rf.train(training_data)

In [ ]:
y_pred2 = []
for i in range(len(testing_data)):
    y_pred2.append(clf_rf.classify_many(testing_data[i]))

In [ ]:
np.savetxt("Y_pred_rf_1.csv", y_pred2, delimiter=",", fmt="%s")